In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("images.tsv", sep='\t', header=None)

In [ ]:
train

In [ ]:
y = train[3072].values

In [ ]:
y

In [ ]:
x = train.drop(columns=[3072]).values

In [ ]:
x.shape

In [ ]:
def draw_img(x):
    plt.figure(figsize=(12, 6))
    for i in range(3):
        plt.subplot(1, 5, i + 1)
        plt.imshow(x[i].reshape(32,32,3));

In [ ]:
x_new = np.zeros_like(x)

In [ ]:
weight = pd.read_csv("wb_model.tsv", sep='\t', header=None)

In [ ]:
def preprocess(x):
    return 0.01 * x - 1.28

In [ ]:
w = weight.values

In [ ]:
w.shape

In [ ]:
def softmax(x, y = None):
    shifted_logits = x - np.max(x)
    Z = np.sum(np.exp(shifted_logits))
    log_probs = shifted_logits - np.log(Z)
    probs = np.exp(log_probs)
    if y != None:
        loss = -log_probs[y]
        dx = probs.copy()
        dx[y] -= 1
        return probs, dx
    return probs

In [ ]:
softmax(preprocess(x[0]) @ w.T)

In [ ]:
def mse(a, b):
    return mean_squared_error(a, b)

In [ ]:
x_metric = [9999.9, 9999.9, 9999.9]

In [ ]:
print(softmax(preprocess(x[0]), 3))

In [ ]:
x_metric[2] =999.9
for i in range(3):
    x_orig = np.copy(x[i])
    yy = y[i]
    x_mod = np.copy(x_orig).astype(float)
    lr = 5
    while True:
        score = softmax(preprocess(x_mod) @ w.T)
        grad = np.copy(score)
        grad[yy] -= 1
        if score[yy] > 0.505:
            print(score)
            break
        dx = grad @ w
        print(np.max(dx), np.min(dx))
        #break
        #llr = -np.sign(dx)
        #x_mod += llr
        x_mod -= (dx * lr)
        x_mod = np.maximum(x_mod, 0)
        x_mod = np.minimum(x_mod, 255)
        #print(x_mod)
        print(score[yy], mse(x_mod, x_orig))
    if x_metric[i] > mse(x_mod, x_orig):
        x_new[i] = np.copy(x_mod)
        x_metric[i] = mse(x_mod, x_orig)

In [ ]:
coeff = 1

for i in range(3):
    x_orig = np.copy(x[i])
    yy = y[i]
    score = lambda q: softmax(preprocess(q) @ w.T)[yy]
    sc = score(x_orig)
    j = 0
    x_mod = np.array(x_orig)
    diff = np.zeros(x_orig.shape[0])
    while True:
        x_modd = np.array(x_mod)
        x_modd[j] = min(255, x_mod[j] + 1)
        scor = score(x_modd)
        a = mse(x_mod, x_orig)
        if scor - sc > coeff*diff.mean():
            diff[j] = scor-sc
            sc = scor
            x_mod[j] += 1
        x_modd[j] = max(0, x_mod[j] - 1)
        scor = score(x_modd)
        if scor - sc > coeff*diff.mean():
            diff[j] = scor-sc
            sc = scor
            x_mod[j] -= 1
        
        if sc > 0.5:
            print("YEAH")
            print(mse(x_mod, x_orig))
            break
        j += 1
        if j == x_orig.shape[0]:
            #rint(diff.max())
            j = 0

    if x_metric[i] > mse(x_mod, x_orig):
        x_new[i] = x_mod
        x_metric[i] = mse(x_mod, x_orig)

In [ ]:
from heapq import heappop, heappush

hyperpar= 1.7
def mymse(x_orig, x_mod, j):
    return (x_orig[j] - x_mod[j]) ** 2

def add_list(diff, x_mod, j, sc):
    x_modd = np.copy(x_mod)
    x_modd[j] = min(255, x_mod[j] + 1)
    scor = score(x_modd)
    if scor - sc > 0:
        heappush(diff, (-(scor-sc)**(1/hyperpar)/(mymse(x_modd, x_orig, j)-mymse(x_mod, x_orig, j)), j, 1))
        #heappush(diff, ((mymse(x_modd, x_orig, j)-mymse(x_mod, x_orig, j))/(scor-sc)**(1/hyperpar), j, 1))
        return
    else:
        x_modd[j] = max(0, x_mod[j] - 1)
        scor = score(x_modd)
        if scor - sc > 0:
            heappush(diff, (-(scor-sc)**(1/hyperpar)/(mymse(x_modd, x_orig, j)-mymse(x_mod, x_orig, j)), j, -1))
            #heappush(diff, ((mymse(x_modd, x_orig, j)-mymse(x_mod, x_orig, j))/(scor-sc)**(1/hyperpar), j, -1))
            return

for i in range(0, 3):
    x_orig = np.copy(x[i])
    yy = y[i]
    score = lambda q: softmax(preprocess(q) @ w.T)[yy]
    sc = score(x_orig)
    x_mod = np.array(x_orig)
    best_mse = 9999
    diff = []
    for j in range(x_orig.shape[0]):
        add_list(diff, x_mod, j, sc)
    while True:
        _, j, _sum = heappop(diff)
        x_mod[j] += _sum
        sc = score(x_mod)
        add_list(diff, x_mod, j, sc)
        if sc > 0.5:
            mmssee = mse(x_mod, x_orig)
            if best_mse > mmssee:
                print("BEST")
                best_mse = mmssee
                best_arr = np.copy(x_mod)
        if sc > 0.6:
            print("0.6!")
            break
    if x_metric[i] > mse(x_mod, x_orig):
        x_new[i] = x_mod
        x_metric[i] = mse(x_mod, x_orig)

In [ ]:
x_metric, x_new

In [ ]:
draw_img(x)

In [ ]:
draw_img(x_new)

In [ ]:
mse(x_new, x)

In [ ]:
np.min(x_new[0] - x[0])

In [ ]:
for i in range(3):
    print(x_new[i])
    print(softmax(preprocess(x_new[i]) @ w.T)[y[i]])

In [ ]:
np.savetxt("task_p_out.txt", x_new, fmt="%d")

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image

In [ ]:
def tpreprocess(x):
    return torch.Tensor(0.01 * x - 1.28)

def tdeprocess(x):
    return (100 * x + 128).data.numpy()

In [ ]:
model = nn.Sequential(nn.Linear(32*32*3, 10, bias=False))
for param in model.parameters():
    param = torch.tensor(w)
    param.requires_grad = False

In [ ]:
def make_fooling_image(X, target_y, model):
    """
    Generate a fooling image that is close to X, but that the model classifies
    as target_y.

    Inputs:
    - X: Input image; Tensor of shape (1, 3, 224, 224)
    - target_y: An integer in the range [0, 1000)
    - model: A pretrained CNN

    Returns:
    - X_fooling: An image that is close to X, but that is classifed as target_y
    by the model.
    """
    # Initialize our fooling image to the input image, and make it require gradient
    X_fooling = X.clone()
    X_fooling = X_fooling.requires_grad_()
    
    learning_rate = 0.1
    ##############################################################################
    # TODO: Generate a fooling image X_fooling that the model will classify as   #
    # the class target_y. You should perform gradient ascent on the score of the #
    # target class, stopping when the model is fooled.                           #
    # When computing an update step, first normalize the gradient:               #
    #   dX = learning_rate * g / ||g||_2                                         #
    #                                                                            #
    # You should write a training loop.                                          #
    #                                                                            #
    # HINT: For most examples, you should be able to generate a fooling image    #
    # in fewer than 100 iterations of gradient ascent.                           #
    # You can print your progress over iterations to check your algorithm.       #
    ##############################################################################
    
    for _ in range(10000):
        scores = model(X_fooling)
        print(softmax(X_fooling.clone().data.numpy() @ w.T)[target_y])
        if softmax(X_fooling.data.numpy() @ w.T)[target_y] > 0.5:
            break
        loss = scores[target_y]
        loss.backward()
        dx = learning_rate * X_fooling.grad #/ X_fooling.grad.norm()
        with torch.no_grad():
            X_fooling += dx
            X_fooling.grad.zero_()
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    return X_fooling

In [ ]:
idx = 0
target_y = 3
ndx = x[0]

X_tensor = tpreprocess(ndx)
X_fooling = make_fooling_image(X_tensor, target_y, model)
scores = model(X_fooling)

In [ ]:
def deprocess(x):
    return (x + 1.28) * 100

In [ ]:
X_fooling_np = tdeprocess(X_fooling.clone())
X_fooling_np = np.asarray(X_fooling_np.data).astype(np.uint8)

In [ ]:
X_fooling_np

In [ ]:
plt.imshow(X_fooling_np.reshape(32,32,3));

In [ ]:
mse(X_fooling_np,x[0])

In [ ]:
softmax(preprocess(X_fooling_np) @ w.T)[target_y]